Note 1 : This notebook draws heaviily from the tutorial available here : https://www.analyticsvidhya.com/blog/2019/09/introduction-information-extraction-python-spacy/

Note 2 : You should ensure that you have installed SpaCy and a SpaCy model (such as en_core_web_sm) to run the code in this notebook.  You can download the en_core_web_sm by running the command :

python -m spacy download en_core_web_sm

in your terminal or command prompt.

In this notebook, we'll look at how to use Hearst Patterns for recognizing hyponymy using SpaCy.  Hearst Patterns are named after Prof Marti Hearst, Computational Linguistics Professor at University of California, Berkeley).  Hyponymy describes the semantic relationship between a hyponym (a subtype) and a hypernym (a supertype).  For example : "cars", "buses", "vans" would all be hyponyms of the hypernym "vehicle".

The Hearst Patterns are as follows (in each case, X represents the hyponym and Y the hypernym):
- X and other Y (e.g cars and other vehicles)
- X or other Y (e.g. cars or other vehicles)
- Y such as X (e.g. vehicles such as cars)
- such Y as X (e.g. such vehicles as cars)
- Y including X (e.g. vehicles including cars)
- Y, especially X (e.g. vehciles, especially cars)

Let's first import the libraries we'll need, and load our SpaCy model :

In [ ]:
import spacy

from spacy.matcher import Matcher
from spacy import displacy

import en_core_web_sm

# Load spacy model
nlp = en_core_web_sm.load()

Let's consider one of the Hearst Patterns - Y such as X, and an example piece of text that contains such a pattern.

In [ ]:
text = ("GDP in developing countries such as Vietnam will continue growing at a high rate")

Let's first apply our SpaCy model to the text, so it can do it's parsing magic!

In [ ]:
doc = nlp(text)

Next let's print the tokens, dependency tags and POS (parts of speech) tags that SpaCy parsed in the text.

Tokens are the individual words or punctuation marks.

Dependency tags describe how words are connected to each other (e.g. "amod" is an "adjectival modifier" and therefore indicates that the role the word plays in the given sentence is to modify an adjective).  You can use spacy.explain("X") where X is the POS Tag or Dependency Tag you want the grammatical name for (so, for example, spacy.explain("amod") will output 'adjectival modifier'.  Of course, unless you're a grammar buff, you'll probably need to Google what many of these mean :)

POS Tags are Parts of Speech tags.  Every word in the English Language belongs to a part of Speech (Nouns, Adverbs etc), of which there are 9 in the English Language.  The POS Tag indicates which one.

In [ ]:
for tok in doc:
    print (f"{tok.text} --> {tok.dep_} --> {tok.pos_}")

Let's have a look at SpaCy's explanation of a few of one of these tags so we can see how spacy.explain works :

In [ ]:
spacy.explain('nsubj')

Now we know what the structure of the sentence looks like in terms of both POS and Depedency, we can try creating a pattern to extract the hyponymy.  We define the pattern as a list of dictionaries, where each dictionary contains a key indicating the name of the option we want to use, and the value representing the value for that option.  The dictionary may contain more than one key:value pair, if more than one option needs to be set for a particular part of the pattern (a token here).  Therefore, each dictionary represents a token that forms part of the pattern.  So a list of three dictionaries would describe a pattern of three tokens.

In the code below, DEP means a Dependency Tag, POS a POS Tag, LOWER the lowercase form of the token's text, and OP an operator or quantififer.  'PROPN' means a proper noun, 'amod' an adjectival modifier, and ? means this thing can occur once or not at all.  See https://spacy.io/usage/rule-based-matching for more details.

So, the pattern below says to look for an optional adjectival modifier, followed by a noun, followed by the words "such as", followed by a proper noun.

In [ ]:
pattern = [{'DEP':'amod', 'OP':"?"}, 
           {'POS':'NOUN'},
           {'LOWER':'such'},
           {'LOWER':'as'},
           {'POS':'PROPN'}]

Now we've got a pattern set up, we can start thinking about how we extract the pattern from the text.  We'll first create a Matcher object.  A Matcher is a SpaCy object which, given a list of patterns, will return the start and end points of those patterns in the text.

To set up a matcher, we first pass in the vocabulary from our loaded SpaCy model, and then add to the matcher an ID for the matcher, and a list of patterns (we've only got one pattern here, but we still need to pass it in as a list).  We then apply the matcher to the document (remember, the document represents the parsed text after the SpaCy model has been applied).

In [ ]:
matcher = Matcher(nlp.vocab)
matcher.add("matching_1", [pattern])
matches = matcher(doc)

The matcher returns a list of three-element tuples, in which each tuple is (match_id, start, end).  match_id is the hash value of the ID of the matcher ("matching_1" in this case).  Start and End represent the token positions of where the identified match starts and ends (So the first token is 0, second token is 1 etc).  We'll get a tuple returned for every match identified. Here, we know we only have one match, so we can just refer to matches[0], which refers to the first (and only) tuple.  We can then specify the "span" (the matched text) using the start and end token positions stored in the second and third elements of the tuple (so matches[0][1] and matches[0][2] respectively).

In [ ]:
span = doc[matches[0][1]:matches[0][2]]

Now let's print the text of the span to see if we successfully extracted the hyponymy.

In [ ]:
print (span.text)

We can use exactly the same approach for other Hearst Patterns.  Here's an example of extracting hyponymy that uses the "X and/or Y" pattern.

In [ ]:
# Hearst Pattern : X and/or Y
print ("Hearst Pattern X and/or Y")
print ("-------------------------")

doc = nlp("Here is how you can keep your car and other vehicles clean.")

# Print tokens, dependencies, POS tags
for tok in doc:
    print (f"{tok.text} --> {tok.dep_} --> {tok.pos_}")
    
print ()

# Define the pattern
pattern = [{'DEP':'amod', 'OP':"?"},
           {'POS':'NOUN'},
           {'LOWER':'and', 'OP':"?"},
           {'LOWER':'or', 'OP':"?"},
           {'LOWER':'other'},
           {'POS':'NOUN'}]

# Create a Matcher object
matcher = Matcher(nlp.vocab)

# Define the Matcher object
matcher.add("matching_1", [pattern])

# Apply the matcher to the SpaCy document
matches = matcher(doc)

# Grab the span of the matched text and print it
span = doc[matches[0][1]:matches[0][2]]
print (span.text)
